In [41]:
import numpy as np
import mxnet as mx
from collections import namedtuple, Counter
from unidecode import unidecode
from mxnet.io import DataIter

import operator
import re

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:36:45.109845. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [3]:
Args = namedtuple(
    'Args', 
    ('test load_epoch num_layers num_hidden num_embed bidirectional gpus '
     'kv_store num_epochs optimizer mom wd lr batch_size disp_batches '
     'stack_rnn dropout model_prefix'))

args = Args(
    test=          False,
    load_epoch=    0,
    num_layers=    2,
    num_hidden=    200,
    num_embed=     200,
    bidirectional= False,
    gpus=          '0,1',
    kv_store=      'device',
    num_epochs=    1,
    optimizer=    'adam',
    mom=           0.9,
    wd=           0.00001,
    lr = 0.001,
    batch_size=    32,
    disp_batches=  50,
    stack_rnn=     False,
    dropout=       0.5,
    model_prefix= 'foo'
)

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:13:07.901140. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [7]:
# Do a word count to get the number of words 

start_label = 1
invalid_label = 0

# Decode text as UTF-8
# Remove diacritical signs and convert to Latin alphabet
# Separate punctuation as separate "words"
def tokenize_text(fname, vocab=None, invalid_label=-1, start_label=0, sep_punctuation=True):
    lines = unidecode(open(fname).read().decode('utf-8')).split('\n')
    lines = [x for x in lines if x]
    lines = map(lambda x: re.findall(r"\w+|[^\w\s]", x, re.UNICODE), lines)    
    sentences, vocab = mx.rnn.encode_sentences(lines, vocab=vocab, invalid_label=invalid_label, start_label=start_label)
    return sentences, vocab

def get_data(src_path, targ_path, start_label=1, invalid_label=0):
    src_sent, src_vocab = tokenize_text(src_path, start_label=start_label,
                                invalid_label=invalid_label)
    
    new_start = max(src_vocab.values())
    
    targ_sent, targ_vocab = tokenize_text(targ_path, start_label=new_start+1,
                                          invalid_label=invalid_label)

    return src_sent, src_vocab, targ_sent, targ_vocab

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:13:50.158607. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [15]:
src_sent, src_vocab, targ_sent, targ_vocab = \
    get_data(
        src_path='./data/europarl-v7.pl-en.en', 
        targ_path='./data/europarl-v7.pl-en.pl',
        start_label=1,
        invalid_label=0
    )

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:23:20.361504. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [29]:
def gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=None):
    length_pairs = map(lambda x: (len(x[0]), len(x[1])), zip(src_sent, targ_sent))
    counts = list(Counter(length_pairs).items())
    c_sorted = sorted(counts, key=operator.itemgetter(0, 1))
    buckets = [i for i in c_sorted if i[1] >= filter_smaller_counts_than]
    return buckets

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:31:03.459087. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [30]:
all_buckets = gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=None)

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:31:05.504688. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [31]:
bucket_filter = 32
batched_buckets = gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=bucket_filter)

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:31:07.104721. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [40]:
num_sent = lambda sent: reduce(lambda a, b: a + b, map(lambda x: x[1], sent))
num_all = num_sent(all_buckets)
num_batched = num_sent(batched_buckets)
print("# of all buckets: %d (# sent: %d)" % (len(all_buckets), num_all))
print("# of buckets with counts < %d filtered out: %d (num sent: %d)" % (bucket_filter, len(batched_buckets), num_batched))

# of all buckets: 9024 (# sent: 629529)
# of buckets with counts < 32 filtered out: 2992 (num sent: 591527)


/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:34:53.229550. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [58]:
def print_sorted_buckets(buckets):
    b = sorted(buckets, key=operator.itemgetter(0, 1))
    for i in b:
        print(i)

print_sorted_buckets(batched_buckets)

((2, 2), 422)
((2, 3), 37)
((2, 5), 45)
((2, 6), 55)
((2, 7), 55)
((2, 8), 75)
((2, 9), 92)
((2, 10), 99)
((2, 11), 91)
((2, 12), 95)
((2, 13), 105)
((2, 14), 100)
((2, 15), 118)
((2, 16), 120)
((2, 17), 105)
((2, 18), 124)
((2, 19), 101)
((2, 20), 125)
((2, 21), 101)
((2, 22), 91)
((2, 23), 93)
((2, 24), 96)
((2, 25), 93)
((2, 26), 77)
((2, 27), 93)
((2, 28), 86)
((2, 29), 63)
((2, 30), 80)
((2, 31), 69)
((2, 32), 56)
((2, 33), 64)
((2, 34), 46)
((2, 35), 67)
((2, 36), 57)
((2, 37), 48)
((2, 39), 41)
((2, 40), 39)
((3, 2), 66)
((3, 3), 59)
((3, 4), 42)
((3, 5), 42)
((3, 6), 67)
((3, 7), 56)
((3, 8), 84)
((3, 9), 95)
((3, 10), 84)
((3, 11), 90)
((3, 12), 106)
((3, 13), 108)
((3, 14), 97)
((3, 15), 100)
((3, 16), 113)
((3, 17), 107)
((3, 18), 102)
((3, 19), 101)
((3, 20), 112)
((3, 21), 87)
((3, 22), 78)
((3, 23), 93)
((3, 24), 97)
((3, 25), 82)
((3, 26), 82)
((3, 27), 72)
((3, 28), 62)
((3, 29), 60)
((3, 30), 60)
((3, 31), 72)
((3, 32), 55)
((3, 33), 73)
((3, 34), 47)
((3, 35), 43)
((3

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:47:56.880658. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [ ]:
# TODO:
# 1) Filter out sentence pairs of which there are fewer than the batch size?
# 2) Just use pairs from 0 to 60 in increments of 5?

In [43]:
class BucketSeq2SeqIter(DataIter):
    """Simple bucketing iterator for sequence-to-sequence models.

    Parameters
    ----------
    sentences : list of list of pairs of int
        encoded sentences (source seq dict int, target seq dict int)
    batch_size : int
        batch_size of data    
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of pairs of int (source seq length, target seq length)
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data    print(buckets)
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, input_seq, output_seq, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC'):
        super(BucketSentenceIter, self).__init__()
        if not buckets:
            buckets = [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
                       if j >= batch_size]
        buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            buck = bisect.bisect_left(buckets, len(sent))
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:38:07.149765. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


((2, 2), 422)
((2, 3), 37)
((2, 5), 45)
((2, 6), 55)
((2, 7), 55)
((2, 8), 75)
((2, 9), 92)
((2, 10), 99)
((2, 11), 91)
((2, 12), 95)
((2, 13), 105)
((2, 14), 100)
((2, 15), 118)
((2, 16), 120)
((2, 17), 105)
((2, 18), 124)
((2, 19), 101)
((2, 20), 125)
((2, 21), 101)
((2, 22), 91)
((2, 23), 93)
((2, 24), 96)
((2, 25), 93)
((2, 26), 77)
((2, 27), 93)
((2, 28), 86)
((2, 29), 63)
((2, 30), 80)
((2, 31), 69)
((2, 32), 56)
((2, 33), 64)
((2, 34), 46)
((2, 35), 67)
((2, 36), 57)
((2, 37), 48)
((2, 39), 41)
((2, 40), 39)
((3, 2), 66)
((3, 3), 59)
((3, 4), 42)
((3, 5), 42)
((3, 6), 67)
((3, 7), 56)
((3, 8), 84)
((3, 9), 95)
((3, 10), 84)
((3, 11), 90)
((3, 12), 106)
((3, 13), 108)
((3, 14), 97)
((3, 15), 100)
((3, 16), 113)
((3, 17), 107)
((3, 18), 102)
((3, 19), 101)
((3, 20), 112)
((3, 21), 87)
((3, 22), 78)
((3, 23), 93)
((3, 24), 97)
((3, 25), 82)
((3, 26), 82)
((3, 27), 72)
((3, 28), 62)
((3, 29), 60)
((3, 30), 60)
((3, 31), 72)
((3, 32), 55)
((3, 33), 73)
((3, 34), 47)
((3, 35), 43)
((3

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:42:03.363440. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [57]:
print(batched_buckets)

[((60, 8), 51), ((16, 8), 398), ((23, 20), 603), ((7, 11), 265), ((28, 28), 368), ((67, 14), 46), ((55, 17), 99), ((36, 49), 45), ((21, 32), 341), ((36, 17), 373), ((3, 21), 87), ((41, 24), 237), ((58, 21), 62), ((37, 50), 61), ((40, 50), 40), ((30, 9), 351), ((38, 53), 39), ((24, 25), 469), ((58, 8), 50), ((36, 9), 274), ((28, 56), 40), ((19, 15), 611), ((67, 20), 36), ((19, 10), 470), ((16, 5), 254), ((33, 19), 441), ((25, 10), 453), ((39, 17), 288), ((20, 53), 44), ((14, 22), 537), ((40, 8), 207), ((2, 33), 64), ((45, 10), 183), ((55, 21), 110), ((55, 25), 70), ((20, 20), 637), ((23, 59), 32), ((11, 6), 263), ((46, 4), 38), ((25, 49), 84), ((46, 15), 183), ((43, 15), 249), ((49, 13), 140), ((21, 13), 618), ((23, 14), 598), ((31, 57), 34), ((44, 29), 152), ((35, 27), 266), ((33, 41), 92), ((15, 36), 248), ((45, 38), 66), ((9, 28), 267), ((18, 14), 648), ((15, 16), 561), ((51, 22), 115), ((10, 43), 83), ((26, 44), 116), ((39, 28), 203), ((2, 15), 118), ((53, 18), 116), ((18, 24), 510)

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:47:12.071457. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [55]:
buckets = list(range(5, 60, 5))
#[10, 20, 30, 40, 50, 60]

                 data_name='data', label_name='softmax_label', dtype='float32',
start_label = 1
invalid_label = 0

def tokenize_text(fname, vocab=None, invalid_label=-1, start_label=0):
    lines = open(fname).readlines()
    lines = [filter(None, i.split(' ')) for i in lines]
    sentences, vocab = mx.rnn.encode_sentences(lines, vocab=vocab, invalid_label=invalid_label, start_label=start_label)
    return sentences, vocab

def get_data(layout):
    train_sent, vocab = tokenize_text("./data/ptb.train.txt", start_label=start_label,
                                      invalid_label=invalid_label)
    val_sent, _ = tokenize_text("./data/ptb.test.txt", vocab=vocab, start_label=start_label,
                                invalid_label=invalid_label)

    data_train  = mx.rnn.BucketSentenceIter(train_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label, layout=layout)
    data_val    = mx.rnn.BucketSentenceIter(val_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label, layout=layout)
    
    print("default: %s" % data_train.default_bucket_key)
    return data_train, data_val, vocab


def train(args):
    data_train, data_val, vocab = get_data('TN')
    if args.stack_rnn:
        stack = mx.rnn.SequentialRNNCell()
        for layer in range(args.num_layers):
            dropout = 0.0
            if layer < (args.num_layers - 1):
                dropout = args.dropout
            stack.add(mx.rnn.FusedRNNCell(args.num_hidden, num_layers=1,
                    mode='lstm', prefix='lstm_%d'%layer, dropout=dropout,
                    bidirectional=args.bidirectional))
        cell = stack
    else:
        cell = mx.rnn.FusedRNNCell(args.num_hidden, num_layers=args.num_layers, dropout=args.dropout,
                mode='lstm', bidirectional=args.bidirectional)

    def sym_gen(seq_len):
        data = mx.sym.Variable('data')
        label = mx.sym.Variable('softmax_label')
        embed = mx.sym.Embedding(data=data, input_dim=len(vocab), output_dim=args.num_embed,name='embed')

        output, _ = cell.unroll(seq_len, inputs=embed, merge_outputs=True, layout='TNC')

        pred = mx.sym.Reshape(output,
                shape=(-1, args.num_hidden*(1+args.bidirectional)))
        pred = mx.sym.FullyConnected(data=pred, num_hidden=len(vocab), name='pred')

        label = mx.sym.Reshape(label, shape=(-1,))
        pred = mx.sym.SoftmaxOutput(data=pred, label=label, name='softmax')

        return pred, ('data',), ('softmax_label',)

    if args.gpus:
        contexts = [mx.gpu(int(i)) for i in args.gpus.split(',')]
    else:
        contexts = mx.cpu(0)

    model = mx.mod.BucketingModule(
        sym_gen             = sym_gen,
        default_bucket_key  = data_train.default_bucket_key,
        context             = contexts)

    if args.load_epoch:
        _, arg_params, aux_params = mx.rnn.load_rnn_checkpoint(
            cell, args.model_prefix, args.load_epoch)
    else:
        arg_params = None
        aux_params = None

    opt_params = {
      'learning_rate': args.lr,
      'wd': args.wd
    }

    if args.optimizer not in ['adadelta', 'adagrad', 'adam', 'rmsprop']:
        opt_params['momentum'] = args.mom

    model.fit(
        train_data          = data_train,
        eval_data           = data_val,
        eval_metric         = mx.metric.Perplexity(invalid_label),
        kvstore             = args.kv_store,
        optimizer           = args.optimizer,
        optimizer_params    = opt_params, 
        initializer         = mx.init.Xavier(factor_type="in", magnitude=2.34),
        arg_params          = arg_params,
        aux_params          = aux_params,
        begin_epoch         = args.load_epoch,
        num_epoch           = args.num_epochs,
        batch_end_callback  = mx.callback.Speedometer(args.batch_size, args.disp_batches),
        epoch_end_callback  = mx.rnn.do_rnn_checkpoint(cell, args.model_prefix, 1)
                              if args.model_prefix else None)

def test(args):
    assert args.model_prefix, "Must specifiy path to load from"
    _, data_val, vocab = get_data('NT')

    if not args.stack_rnn:
        stack = mx.rnn.FusedRNNCell(args.num_hidden, num_layers=args.num_layers,
                mode='lstm', bidirectional=args.bidirectionclass BucketSentenceIter(DataIter):
    """Simple bucketing iterator for language model.
    Label for each step is constructed from data of
    next step.

    Parameters
    ----------
    sentences : list of list of int
        encoded sentences
    batch_size : int
        batch_size of data
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of int
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, sentences, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC'):
        super(BucketSentenceIter, self).__init__()
        if not buckets:
            buckets = [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
                       if j >= batch_size]
        buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            buck = bisect.bisect_left(buckets, len(sent))
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])al).unfuse()
    else:
        stack = mx.rnn.SequentialRNNCell()
        for i in range(args.num_layers):
            cell = mx.rnn.LSTMCell(num_hidden=args.num_hidden, prefix='lstm_%dl0_'%i)
            if args.bidirectional:
                cell = mx.rnn.BidirectionalCell(
                        cell,
                        mx.rnn.LSTMCell(num_hidden=args.num_hidden, prefix='lstm_%dr0_'%i),
                        output_prefix='bi_lstm_%d'%i)
            stack.add(cell)

    def sym_gen(seq_len):
        data = mx.sym.Variable('data')
        label = mx.sym.Variable('softmax_label')
        embed = mx.sym.Embedding(data=data, input_dim=len(vocab),
                                 output_dim=args.num_embed, name='embed')

        stack.reset()
        outputs, states = stack.unroll(seq_len, inputs=embed, merge_outputs=True)

        pred = mx.sym.Reshape(outputs,
                shape=(-1, args.num_hidden*(1+args.bidirectional)))
        pred = mx.sym.FullyConnected(data=pred, num_hidden=len(vocab), name='pred')

        label = mx.sym.Reshape(label, shape=(-1,))
        pred = mx.sym.SoftmaxOutput(data=pred, label=label, name='softmax')

        return pred, ('data',), ('softmax_label',)class BucketSentenceIter(DataIter):
    """Simple bucketing iterator for language model.
    Label for each step is constructed from data of
    next step.

    Parameters
    ----------
    sentences : list of list of int
        encoded sentences
    batch_size : int
        batch_size of data
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of int
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, sentences, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC'):
        super(BucketSentenceIter, self).__init__()
        if not buckets:
            buckets = [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
                       if j >= batch_size]
        buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            buck = bisect.bisect_left(buckets, len(sent))
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])

    if args.gpus:
        contexts = [mx.gpu(int(i)) for i in args.gpus.split(',')]
    else:
        contexts = mx.cpu(0)

    model = mx.mod.BucketingModule(
        sym_gen             = sym_gen,
        default_bucket_key  = data_val.default_bucket_key,
        context             = contexts)
    model.bind(data_val.provide_data, data_val.provide_label, for_training=False)

    # note here we load using SequentialRNNCell instead of FusedRNNCell.
    _, arg_params, aux_params = mx.rnn.load_rnn_checkpoint(stack, args.model_prefix, args.load_epoch)
    model.set_params(arg_params, aux_params)

    model.score(data_val, mx.metric.Perplexity(invalid_label),
                batch_end_callback=mx.callback.Speedometer(args.batch_size, 5))

IndentationError: unexpected indent (<ipython-input-55-51696d919ddc>, line 4)

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:41:51.803097. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [ ]:
import logging
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

# args = parser.parse_args()

if args.num_layers >= 4 and len(args.gpus.split(',')) >= 4 and not args.stack_rnn:
    print('WARNING: stack-rnn is recommended to train complex model on multiple GPUs')

if args.test:
    # Demonstrates how to load a model trained with CuDNN RNN and predict
    # with non-fused MXNet symbol
    test(args)
else:
    train(args)

In [42]:
class BucketSeq2SeqIter(DataIter):
    """Simple bucketing iterator for sequence-to-sequence models.

    Parameters
    ----------
    sentences : list of list of pairs of int
        encoded sentences (source seq dict int, target seq dict int)
    batch_size : int
        batch_size of data    
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of pairs of int (source seq length, target seq length)
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data    print(buckets)
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, input_seq, output_seq, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC'):
        super(BucketSentenceIter, self).__init__()
        if not buckets:
            buckets = [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
                       if j >= batch_size]
        buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            buck = bisect.bisect_left(buckets, len(sent))
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 22:37:22.066196. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
